# Image umming

Goal: Define movies & localization files of multiple measurements, cut patches and randomly sum them.

Localization file:
- Picasso localization file csv, header = frame, x [nm], y [nm], sigma [nm], intensity [photon], offset [photon], bkgstd [photon], uncertainty_xy [nm]
- DeepSTORM2D file format, header = ,frame,x [nm],y [nm],Photon #,Sigma [nm]

Movie: Tif file with one-multiple frames.

Save: Randomly summed patches as tif movie, each patch is a frame and corresponding localization file as csv in DeepSTORM2D format.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import matplotlib.patches as plt_patches
import warnings
warnings.filterwarnings("ignore")
from ImageBinner.widgets import widgetsImageBinner, loadImageBinner
from ImageBinner.tools import imageBinner
from ImageBinner.save import saveImageBinner

### Load

Define the paths to the movie (tif) and localization (csv) files of multiple measurements. The number of loading measurement widgets can be adjusted with "n_measurements" in the code. Empty widgets are ignored.

In [2]:
n_measurements = 2  # vary the number of measurements here!
movie_widgets, localization_widgets = [], []
for i in range(1, n_measurements+1):
    print("measurement", i)
    movie_widget = widgetsImageBinner.DefineFiles("movie", title="Import *.tif file", filetype=("tif files", "*.tif"), idx=i)
    loc_widget = widgetsImageBinner.DefineFiles("localizations", title="Import *.csv file", filetype=("csv files", "*.csv"), idx=i)
    movie_widget.file_button.on_click(movie_widget.open_file)
    movie_widget.file_text_box.observe(movie_widget.change_file_path_box)
    loc_widget.file_button.on_click(loc_widget.open_file)
    loc_widget.file_text_box.observe(loc_widget.change_file_path_box)
    movie_widgets.append(movie_widget)
    localization_widgets.append(loc_widget)
    display(movie_widget.file_text_box, movie_widget.file_button, loc_widget.file_text_box, loc_widget.file_button)

measurement 1


Text(value='', description='Path to movie 1', placeholder='Insert path', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations 1', placeholder='Insert path', style=DescriptionStyle(descri…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

measurement 2


Text(value='', description='Path to movie 2', placeholder='Insert path', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

Text(value='', description='Path to localizations 2', placeholder='Insert path', style=DescriptionStyle(descri…

Button(description='browse', style=ButtonStyle(), tooltip='browse for file')

### Parameters

In [3]:
widget_parameters = widgetsImageBinner.Parameters(107, 100, 30, 200, 100, 10, 2, 1, n_measurements)  # adjust the default parameters
display(widget_parameters.pixel_size, widget_parameters.camera_noise, widget_parameters.patch_size, widget_parameters.n_patches, widget_parameters.n_binned_patches, widget_parameters.bin_size, widget_parameters.min_emitters_per_patch)

Text(value='107', description='Pixel size [nm]', placeholder='Insert pixel size', style=DescriptionStyle(descr…

Text(value='100', description='Camera offset [px intensity]', placeholder='Insert average camera offset', styl…

Text(value='30', description='Patch size [px]', placeholder='Insert k', style=DescriptionStyle(description_wid…

Text(value='200', description='Number of patches created', placeholder='Insert bin size', style=DescriptionSty…

Text(value='100', description='Number of summed patches created', placeholder='Insert the amount of summed pat…

Text(value='10', description='Number of images per patch', placeholder='Insert the amount of images to be summ…

Text(value='2', description='Min emitters per patch', placeholder='Min emitters per patch to be used in summin…

## Input

In [4]:
def scroll_in_time(frame, show_single=True):
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.tight_layout()
    movie_idx = int(widget_parameters.display_movie_idx.value)-1
    fig.set_size_inches(14, 14, forward=True)
    ax1.imshow(data.movies[movie_idx][frame-1], interpolation="nearest", cmap = "gray")
    ax2.imshow(data.movies[movie_idx][frame-1], interpolation="nearest", cmap = "gray")
    x, y = imageBinner.xy_loc_vis(data.loc_files[movie_idx][data.loc_files[movie_idx]["frame"] == frame], int(widget_parameters.pixel_size.value))
    ax2.scatter(x=x, y=y, marker="x")
    ax2.set_xlim([-0.5, data.movies[movie_idx].shape[1]-0.5])
    ax2.set_ylim([data.movies[movie_idx].shape[2]-0.5, -0.5])
    ax1.axis("off"); ax2.axis("off")
    plt.show()
    if show_single:
        display(data.loc_files[movie_idx][data.loc_files[movie_idx]["frame"] == frame])

def run_input(event):
    widget_run.create_clear_output()
    display(widget_parameters.display_movie_idx, widget_run.run_input)
    movie_dirs = [widget.file_text_box.value for widget in movie_widgets if widget.file_text_box.value]
    loc_dirs = [widget.file_text_box.value for widget in localization_widgets if widget.file_text_box.value]
    data.load_data(movie_dirs[:min(len(movie_dirs), len(loc_dirs))], loc_dirs[:min(len(movie_dirs), len(loc_dirs))])
    movie_idx = int(widget_parameters.display_movie_idx.value)-1
    movies_dim = imageBinner.get_dimensions(data.movies)
    if data.loc_files and data.movies:
        print("Movie " + str(movie_idx+1) + " dimensions: " + str(movies_dim[movie_idx][1])+"x"+str(movies_dim[movie_idx][2])+" with "+str(movies_dim[movie_idx][0])+" frames\n")
        print("Choose a frame with its localizations to display:")
        if movies_dim[movie_idx][0] > 1:
            interact(scroll_in_time, frame=widgets.IntSlider(min=1, max=movies_dim[movie_idx][0], step=1, value=0, continuous_update=False))
        else:
            fig = plt.figure(figsize=(6,6))
            plt.imshow(data.movies[movie_idx], interpolation="nearest", cmap = "gray")
            plt.title("Training source")
            plt.axis("off")
        print("Input densities: ")
        imageBinner.vis_input_density(movies_dim, data.loc_files, int(widget_parameters.pixel_size.value))
    else:
        print("Please define the files of at least one measurement (movie-tif, localizations-csv).")

data = loadImageBinner.Data()
widget_run = widgetsImageBinner.RunAnalysis()
display(widget_parameters.display_movie_idx, widget_run.run_input)
widget_run.run_input.on_click(run_input)

Dropdown(description='Measurement', options=(1, 2), style=DescriptionStyle(description_width='initial'), value…

Button(description='load & display', style=ButtonStyle(), tooltip='display input data')

## Patches

In [5]:
def scroll_in_time_patch(frame, show_single=True):
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.tight_layout()
    fig.set_size_inches(12, 12, forward=True)
    ax1.imshow(patches.random_patches_binned[frame-1], interpolation="nearest", cmap = "gray")
    ax2.imshow(patches.random_patches_binned[frame-1], interpolation="nearest", cmap = "gray")
    x, y = imageBinner.xy_loc_vis(patches.random_locs_binned[frame-1], int(widget_parameters.pixel_size.value))
    ax2.scatter(x=x, y=y, marker="x")
    ax2.set_xlim(-0.5, int(widget_parameters.patch_size.value)-0.5)
    ax2.set_ylim(int(widget_parameters.patch_size.value)-0.5, -0.5)
    ax1.axis("off"); ax2.axis("off")
    plt.show()
    display(patches.random_locs_binned[frame-1])
    if show_single:
        show_single_frames(frame)
        
def show_single_frames(frame):
    patch_idx = patches.random_patches_binned_idx[frame-1]   
    for i in patch_idx:
        fig, (ax1, ax2, ax3) = plt.subplots(1,3)
        fig.tight_layout()
        fig.set_size_inches(10, 10, forward=True)
        ax1.imshow(patches.random_patches[i], interpolation="nearest", cmap = "gray")
        # single patch
        x, y = imageBinner.xy_loc_vis(patches.random_locs[i], int(widget_parameters.pixel_size.value))
        ax1.set_xlim(-0.5, int(widget_parameters.patch_size.value)-0.5)
        ax1.set_ylim(int(widget_parameters.patch_size.value)-0.5, -0.5)
        ax1.scatter(x=x, y=y, marker="x")
        # final patch (=summed image)
        ax2.imshow(patches.random_patches_binned[frame-1], interpolation="nearest", cmap = "gray")
        x, y = imageBinner.xy_loc_vis(patches.random_locs_binned[frame-1], int(widget_parameters.pixel_size.value))
        ax2.set_xlim(-0.5, int(widget_parameters.patch_size.value)-0.5)
        ax2.set_ylim(int(widget_parameters.patch_size.value)-0.5, -0.5)
        ax2.scatter(x=x, y=y, marker="x")
        # full frame + window
        movie_idx = patches.random_patches_idx[i][0] - 1
        frame_idx = patches.random_patches_idx[i][1]
        ax3.imshow(data.movies[movie_idx][frame_idx-1], interpolation="nearest", cmap = "gray")
        rect = plt_patches.Rectangle((patches.random_patch_nm[i][1], patches.random_patch_nm[i][0]), int(widget_parameters.patch_size.value), int(widget_parameters.patch_size.value), linewidth=1, edgecolor='darkorange', facecolor='none')
        ax3.add_patch(rect)
        x, y = imageBinner.xy_loc_vis(data.loc_files[movie_idx][data.loc_files[movie_idx]["frame"] == frame_idx], int(widget_parameters.pixel_size.value))
        ax3.set_xlim([-0.5, data.movies[movie_idx].shape[1]-0.5])
        ax3.set_ylim([data.movies[movie_idx].shape[2]-0.5, -0.5])
        ax3.scatter(x=x, y=y, marker="x")
        ax1.axis("off"); ax2.axis("off"); ax3.axis("off")
        plt.show()
        display(patches.random_locs[i])

def run_patches(event):
    widget_run.create_clear_output()
    display(widget_run.run_patches)
    patches.create_patches(int(widget_parameters.n_patches.value), data.movies, data.loc_files,
                           int(widget_parameters.patch_size.value), int(widget_parameters.n_binned_patches.value),
                           int(widget_parameters.bin_size.value), int(widget_parameters.min_emitters_per_patch.value),
                           int(widget_parameters.pixel_size.value), int(widget_parameters.camera_noise.value),
                           augmentation=False)
    print("Patch dimensions: " + str(patches.random_patches_binned[0].shape[0])+"x"+str(patches.random_patches_binned[0].shape[1])+" with "+str(len(patches.random_patches_binned))+" frames\n")
    print("Choose a patch with its localizations to display (1 patch = 1 frame):")
    interact(scroll_in_time_patch, frame=widgets.IntSlider(min=1, max=patches.random_patches_binned.shape[0], step=1, value=0, continuous_update=False))
    imageBinner.vis_patch_density(patches.random_locs_binned, int(widget_parameters.patch_size.value), int(widget_parameters.pixel_size.value))

patches = imageBinner.GetPatches()
display(widget_run.run_patches)
widget_run.run_patches.on_click(run_patches)

Button(description='create', style=ButtonStyle(), tooltip='create patches')

### Save

The created patches and corresponding localization file will be saved as *.tif and *.csv under the chosen directory.

In [6]:
widget_save = widgetsImageBinner.SaveResults()
display(widget_save.box_filename, widget_save.dir_box, widget_save.dir_button)
widget_save.dir_button.on_click(widget_save.open_dir)
widget_save.dir_box.observe(widget_save.change_dir_box)

Text(value='', description='Filename', placeholder='name of folder', style=DescriptionStyle(description_width=…

Text(value='', description='Directory', placeholder='Directory to save', style=DescriptionStyle(description_wi…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [7]:
display(widget_save.save_button)
def save_analysis(event):
    widget_save.create_clear_output()
    display(widget_save.save_button)   
    saveImageBinner.save(widget_save.dir_box.value, widget_save.box_filename.value, patches.random_patches_binned, patches.random_locs_binned)
    print("Results are saved.")
    
widget_save.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save the results')